In [1]:
# Operações para Silver

from pyspark.sql import *
from delta import *

builder = SparkSession.builder.appName("topicos").config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension").config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
spark = configure_spark_with_delta_pip(builder).getOrCreate()

INPUT_PATH = "/home/felipe/code/topicos_dados/dados/"
BRONZE_PATH = "/home/felipe/code/deltalake/lake/bronze/"
SILVER_PATH = "/home/felipe/code/deltalake/lake/silver/"

print(spark.version)


24/03/14 22:31:03 WARN Utils: Your hostname, desktop resolves to a loopback address: 127.0.1.1; using 192.168.0.106 instead (on interface enp6s0)
24/03/14 22:31:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/felipe/.local/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/felipe/.ivy2/cache
The jars for the packages stored in: /home/felipe/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a6114734-b265-4672-a123-9610c96476a7;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 874ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0 

3.5.0


In [43]:
from pyspark.sql import *
from delta import *

builder = SparkSession.builder.appName("topicos").config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension").config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
spark = configure_spark_with_delta_pip(builder).getOrCreate()


brbus_table = DeltaTable.create(spark) \
    .tableName("silver_brbus") \
    .addColumn("latitude", dataType="DOUBLE") \
    .addColumn("longitude", dataType="DOUBLE") \
    .addColumn("bus_id", dataType="STRING") \
    .addColumn("city", dataType="STRING") \
    .addColumn("bus_code", dataType="STRING") \
    .addColumn("bus_speed", dataType="DOUBLE") \
    .addColumn("bus_direction", dataType="INT") \
    .addColumn("queried_at", dataType="STRING") \
    .addColumn("updated_at", dataType="STRING") \
    .location("/home/felipe/code/topicos_dados/lake/silver/silver_brbus") \
    .execute()



In [42]:
spark.sql("DROP TABLE IF EXISTS silver_brbus")

DataFrame[]

In [3]:
from deltalake import DeltaTable

teste = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_cwb/")
teste.show()

24/03/12 11:24:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+----------+--------------+--------+----------+---------------+------+----------+--------------------+---------+-------------------+-------------+
|  latitude| longitude|  tipo_veiculo|bus_code|  situacao|     situacao_2|tabela|is_adapted|          queried_at|   bus_id|         updated_at|bus_direction|
+----------+----------+--------------+--------+----------+---------------+------+----------+--------------------+---------+-------------------+-------------+
|-25.439138|-49.272285|MICRO ESPECIAL|     762|  ATRASADO|REALIZANDO ROTA|   2-2|         1|2024-01-30T23:19:21Z|CWB_JI859|2024-01-3023:18:00Z|            1|
|-25.442966|-49.245985|MICRO ESPECIAL|     463|NO HORÁRIO|REALIZANDO ROTA|     2|         1|2024-01-30T23:19:21Z|CWB_DN603|2024-01-3023:19:00Z|            2|
|-25.457181|-49.222675|MICRO ESPECIAL|     463|NO HORÁRIO|REALIZANDO ROTA|     4|         1|2024-01-30T23:19:21Z|CWB_DN608|2024-01-3023:19:00Z|            1|
|-25.441758|-49.347495|    ARTICULADO|     040|  ATR

In [44]:
from deltalake import DeltaTable

table_brbus = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_brbus/")
table_brbus.show()

+--------+---------+------+----+--------+---------+-------------+----------+----------+
|latitude|longitude|bus_id|city|bus_code|bus_speed|bus_direction|queried_at|updated_at|
+--------+---------+------+----+--------+---------+-------------+----------+----------+
+--------+---------+------+----+--------+---------+-------------+----------+----------+



In [24]:
rj_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_rj/")
rj_silver.schema

StructType([StructField('latitude', DoubleType(), True), StructField('longitude', DoubleType(), True), StructField('bus_speed', DoubleType(), True), StructField('bus_code', StringType(), True), StructField('queried_at', StringType(), True), StructField('updated_at', StringType(), True), StructField('bus_id', StringType(), True)])

In [27]:
sp_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_sp/")
sp_silver.schema

StructType([StructField('latitude', DoubleType(), True), StructField('longitude', DoubleType(), True), StructField('bus_code', StringType(), True), StructField('station0', StringType(), True), StructField('station1', StringType(), True), StructField('queried_at', TimestampType(), True), StructField('bus_id', StringType(), True), StructField('updated_at', TimestampType(), True)])

In [17]:
cwb_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_cwb/")
cwb_silver.schema

StructType([StructField('latitude', StringType(), True), StructField('longitude', StringType(), True), StructField('tipo_veiculo', StringType(), True), StructField('bus_code', StringType(), True), StructField('situacao', StringType(), True), StructField('situacao_2', StringType(), True), StructField('tabela', StringType(), True), StructField('is_adapted', StringType(), True), StructField('queried_at', StringType(), True), StructField('bus_id', StringType(), True), StructField('updated_at', StringType(), True), StructField('bus_direction', IntegerType(), True)])

In [18]:
bsb_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_bsb/")
bsb_silver.schema

StructType([StructField('longitude', DoubleType(), True), StructField('latitude', DoubleType(), True), StructField('bus_speed', DoubleType(), True), StructField('direcao', DoubleType(), True), StructField('bus_code', StringType(), True), StructField('queried_at', StringType(), True), StructField('updated_at', StringType(), True), StructField('bus_id', StringType(), True), StructField('bus_direction', IntegerType(), True)])

In [45]:
# Adding city values to each DF

from pyspark.sql.functions import lit
# SP

sp_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_sp/")
sp_silver = sp_silver.withColumn("city",lit("Sao Paulo"))
sp_selected = sp_silver.select(["longitude", "latitude","bus_code","queried_at","updated_at","bus_id","city"])


# RJ
rj_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_rj/")
rj_silver = rj_silver.withColumn("city",lit("Rio de Janeiro"))
rj_selected = rj_silver.select(["longitude", "latitude","bus_speed","bus_code","queried_at","updated_at","bus_id","city"])


# CWB
cwb_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_cwb/")
cwb_silver = cwb_silver.withColumn("city",lit("Curitiba"))
cwb_selected = cwb_silver.select(["longitude", "latitude","bus_code","queried_at","updated_at","bus_id","bus_direction","city"])

#BSB

bsb_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_bsb/")
bsb_silver = bsb_silver.withColumn("city",lit("Brasilia"))

bsb_selected = bsb_silver.select(["longitude", "latitude","bus_speed","bus_code","queried_at","updated_at","bus_id","bus_direction","city"])

# Merging into BRBus Table

sp_selected.write.format("delta").mode("append").save("/home/felipe/code/topicos_dados/lake/silver/silver_brbus")
rj_selected.write.format("delta").mode("append").save("/home/felipe/code/topicos_dados/lake/silver/silver_brbus")
cwb_selected.write.format("delta").mode("append").save("/home/felipe/code/topicos_dados/lake/silver/silver_brbus")
bsb_selected.write.format("delta").mode("append").save("/home/felipe/code/topicos_dados/lake/silver/silver_brbus")



24/03/14 23:49:41 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/03/14 23:49:41 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
24/03/14 23:49:41 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
24/03/14 23:49:42 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
24/03/14 23:49:42 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/03/14 23:49:42 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/03/14 23:49:42 WARN MemoryManager: Total allocation exceeds 95,00%

In [46]:
brbus = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_brbus/")
brbus.show()



+----------+----------+---------+--------+--------+---------+-------------+--------------------+-------------------+
|  latitude| longitude|   bus_id|    city|bus_code|bus_speed|bus_direction|          queried_at|         updated_at|
+----------+----------+---------+--------+--------+---------+-------------+--------------------+-------------------+
|-25.435106| -49.27297|CWB_JI859|Curitiba|     762|     NULL|            1|2024-01-30T23:23:21Z|2024-01-3023:23:00Z|
|-25.451611|-49.227308|CWB_DN603|Curitiba|     463|     NULL|            2|2024-01-30T23:23:21Z|2024-01-3023:22:00Z|
|-25.443568| -49.23387|CWB_DN608|Curitiba|     463|     NULL|            1|2024-01-30T23:23:21Z|2024-01-3023:23:00Z|
|-25.436141|-49.348906|CWB_GB611|Curitiba|     040|     NULL|            2|2024-01-30T23:23:21Z|2024-01-3023:22:00Z|
|-25.466733|-49.344451|CWB_HB609|Curitiba|     040|     NULL|            2|2024-01-30T23:23:21Z|2024-01-3023:23:00Z|
|-25.465451|-49.345108|CWB_JB607|Curitiba|     040|     NULL|   